In [ ]:
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta

# Define the important variables here
target_fips = 6019
target_county_name = "Fresno"
target_data_file_name = f"Datasets/merged_tp_precip_wind_fmc_{target_county_name}.csv"

# Use the adjacent county of the target county to find aws data 
# Use the following link to find the adjacent county
# https://gis.data.ca.gov/datasets/8713ced9b78a4abb97dc130a691a8695/explore?location=39.765076%2C-121.456785%2C8.00
referral_fips = 6039
referral_country_name = "Madera"

# Load the target county data
fire_data = pd.read_csv(target_data_file_name)
print(f"The dataset {target_data_file_name} contains (row, column) = ")
print(fire_data.shape)

# Initial the wind csv
wind = pd.read_csv("Datasets/wind_with_fips.csv")

referral_count = wind['county'].str.contains(referral_country_name).sum()
print(f"The referral county {referral_country_name} has {referral_count} rows of data in the wind_with_fips.csv")
    
# If not data found, exit the program
if ( int(referral_count) <= 0):
    #quit()
    raise SystemExit("Stop right there!")

    
# Extract the whole volume 
def filter_dataframe_by_value(df, column_name, value_to_find):
    filtered_rows = df[df[column_name] == value_to_find]
    return filtered_rows

if __name__ == "__main__":

    # Create a new DataFrame containing only rows with the specific value
    wind_referral = filter_dataframe_by_value(wind, 'fips', referral_fips)

# Add the new column here
def merge_dataframes_on_match(df1, df2, df1_column1, df2_column1, df2_column_to_sum1, df2_column_to_sum2):
    for index1, row1 in df1.iterrows():
        row1_datetime = datetime.strptime(str(row1[df1_column1]),  "%Y%m%d")

        row2_datetime = row1_datetime; 
        aws_to_fill = 0.0
        for index2, row2 in df2.iterrows():
            row2_datetime = datetime.strptime(row2[df2_column1], "%Y-%m-%d")
            delta_days = (row1_datetime - row2_datetime).days
            # print(f"Found a row with diff days {delta_days} between {row1_datetime} and {row2_datetime}")
            if (delta_days < 50):
                aws_to_fill = row2[df2_column_to_sum1]
            if (delta_days < 25):
                aws_to_fill = row2[df2_column_to_sum1]
            if (delta_days < 10):
                aws_to_fill = row2[df2_column_to_sum1]
            if (delta_days < 5):
                aws_to_fill = row2[df2_column_to_sum1]
            if (delta_days < 3):
                aws_to_fill = row2[df2_column_to_sum1]
            if (delta_days < 1):
                aws_to_fill = row2[df2_column_to_sum1]
                break   
                
        print(f"Add data to the original {row1_datetime} <- {row2_datetime} : {aws_to_fill}")
        df1.at[index1, df2_column_to_sum1] = aws_to_fill
                
merge_dataframes_on_match(fire_data, wind_referral.sort_values("date"), 'DATE', 'date', 'aws', 'prcp')

# Save the data to its original file
fire_data.to_csv(target_data_file_name)
print(f"Saved to {target_data_file_name}")
